In [10]:
%cd ..
%pwd 

/home/ruslan/src/IndustrialMotionDetection


/home/ruslan/src/IndustrialMotionDetection/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/ruslan/src/IndustrialMotionDetection'

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().cwd().parent.absolute() / "src"))
sys.path

['/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/home/ruslan/src/IndustrialMotionDetection/.venv/lib/python3.9/site-packages',
 '/home/ruslan/src/IndustrialMotionDetection/src']

In [ ]:
import random
import torch
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from tqdm.auto import tqdm
from models.dataset import RandomDatasetDecord
from config import YamlConfigReader
from models.dataset import collate_fn

In [8]:
conf_file_path = 'conf/meva_preproc.yaml'

In [11]:
config_source = YamlConfigReader(conf_file_path)
params = config_source.get_all()
params

{'annotations_csv': 'data/MEVA/meva_processed/annotations.csv',
 'train': 'data/MEVA/meva_processed/train.csv',
 'test': 'data/MEVA/meva_processed/test.csv',
 'annotations_folder': 'data/MEVA/meva-data-repo/annotation/DIVA-phase-2/MEVA/kitware-meva-training',
 'bbox_area_limit': 10000,
 'display_annotations': False,
 'padding_frames': 30,
 'result_folder': 'data/MEVA/meva_processed',
 'split_seed': 42,
 'target_activities': ['person_talks_on_phone',
  'person_texts_on_phone',
  'person_picks_up_object',
  'person_reads_document',
  'person_interacts_with_laptop'],
 'test_size': 0.2,
 'videos_root': 'data/MEVA/mevadata-public-01/drops-123-r13'}

In [26]:
class2idx = {class_name: i for i, class_name in enumerate(params['target_activities'])}
idx2class = {v: k for k, v in class2idx.items()}
class2idx

{'person_talks_on_phone': 0,
 'person_texts_on_phone': 1,
 'person_picks_up_object': 2,
 'person_reads_document': 3,
 'person_interacts_with_laptop': 4}

In [ ]:
image_datasets = dict()
image_datasets['test'] = RandomDatasetDecord(
    params['test'],
    class2idx=class2idx,
    epoch_size_ratio=1.0,
    video_transform=None,
    frame_sample_rate=1.0,
    clip_len=16,
    num_classes=5)

In [15]:
len(image_datasets['test'])

40

In [ ]:
for i, x in tqdm(enumerate(image_datasets['test']), 
                 total=len(image_datasets['test'])):
    if i == 1: break
    print(x['video'].shape)
    print(x['target'])
    print(x['path'])
    print(x['bbox'])
    print()
    pass

  0%|          | 0/40 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/meva/2018-03-15/15/2018-03-15.15-10-00.15-15-00.bus.G331_frange0-5802.avi'

In [27]:
import gc
gc.collect()

11

In [ ]:
seed = 2025

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

dataloaders = dict()
dataloaders['test'] = DataLoader(image_datasets['test'],
                                 batch_size=batch_size,
                                 pin_memory=True,
                                 collate_fn=collate_fn,
                                 worker_init_fn=seed_worker,
                                 generator=g)

In [ ]:
len(dataloaders['test'])

In [ ]:
tqdm_progress = tqdm(enumerate(dataloaders['test']), 
                     total=len(dataloaders['test']))

for i, batch in tqdm_progress:
    if i == 0:
        print(f"{batch['path']=}\n{batch['video'].shape=}")
        first_video = batch['video'][0].permute(1, 0, 2, 3)
        print(f"{first_video.shape=}")
        print(f"pixel range: {first_video.min()} - {first_video.max()}")

        print(f"{len(batch['bbox'])=}")
        bbox_coords = batch['bbox'][0].bbox.tolist()
        print(bbox_coords)

        print(f"{len(batch['target'])=}")

        fig, axes = plt.subplots(2, clip_len//2, figsize=(12, 12))

        for i, ax in enumerate(axes.flat):
            img = first_video[i].permute(1, 2, 0).numpy()
            img = (img - img.min()) / (img.max() - img.min())

            ax.imshow(img)
            ax.axis('off')

            for bbox_coord in bbox_coords:
                x_min, y_min, x_max, y_max = bbox_coord
                width = x_max - x_min
                height = y_max - y_min
                rect = patches.Rectangle((x_min, y_min), width, height,
                                          linewidth=2, edgecolor='red', facecolor='none')
                ax.add_patch(rect)

        plt.tight_layout()
        plt.show()

    break

print(f"Elapsed: {tqdm_progress.format_dict['elapsed']}")